In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import LeaveOneOut, cross_val_score, GroupKFold, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from tabulate import tabulate
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from scipy.fft import fft
import time
from pyentrp import entropy as ent
import warnings
from sklearn.exceptions import ConvergenceWarning
import csv
from sklearn.pipeline import Pipeline

In [2]:
# Function to reduce features using SelectKBest
def select_features(X_train, y_train, X_test, k=10):
    # Initialize and fit SelectKBest
    selector = SelectKBest(score_func=f_classif, k=k)
    selector.fit(X_train, y_train)
    
    # Transform both training and testing data
    X_train_reduced = selector.transform(X_train)
    X_test_reduced = selector.transform(X_test)
    
    return X_train_reduced, X_test_reduced

In [4]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

def run_mlp_grid_search(features, labels):
    pipe = Pipeline([
        ('select_k_best', SelectKBest(score_func=f_classif)),
        ('mlp', MLPClassifier(solver='adam', activation='tanh', alpha=0.3))
    ])

    # Definir rangos para 'k' con saltos de 5 en 5
    k_range = list(range(10, min(36, features.shape[1] + 1), 1))

    params = {
        'select_k_best__k': k_range,
        'mlp__hidden_layer_sizes': [(100,), (150,), (200,)],
        'mlp__alpha': [0.01, 0.1, 0.3],
        'mlp__solver': ['adam'],
        'mlp__activation': ['relu'],
        'mlp__early_stopping': [True],
        'mlp__validation_fraction': [0.1],
        'mlp__n_iter_no_change': [20]
    }

    grid_search = GridSearchCV(pipe, param_grid=params, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(features, labels)
    
    print("Optimized parameters for MLP:", grid_search.best_params_)
    print("Optimized cross-validation score: {:.2f}".format(grid_search.best_score_))

def process_file(file_path):
    df = pd.read_csv(file_path)
    labels = df['label']
    features = df.drop(columns=['subject_id', 'label'])
    run_mlp_grid_search(features, labels)

def main(folder_path):
    import os
    for file_name in os.listdir(folder_path):
        if file_name.startswith("features_") and file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)
            print(f"\nProcessing file: {file_name}")
            process_file(file_path)

# Cambia la ruta según corresponda
main('/home/ximo/Escritorio/ProyectoTFG/featuresExtended')



Processing file: features_1.csv


Optimized parameters for MLP: {'mlp__activation': 'relu', 'mlp__alpha': 0.01, 'mlp__early_stopping': True, 'mlp__hidden_layer_sizes': (100,), 'mlp__n_iter_no_change': 20, 'mlp__solver': 'adam', 'mlp__validation_fraction': 0.1, 'select_k_best__k': 31}
Optimized cross-validation score: 0.33

Processing file: features_4.csv
Optimized parameters for MLP: {'mlp__activation': 'relu', 'mlp__alpha': 0.3, 'mlp__early_stopping': True, 'mlp__hidden_layer_sizes': (150,), 'mlp__n_iter_no_change': 20, 'mlp__solver': 'adam', 'mlp__validation_fraction': 0.1, 'select_k_best__k': 12}
Optimized cross-validation score: 0.37

Processing file: features_3.csv
Optimized parameters for MLP: {'mlp__activation': 'relu', 'mlp__alpha': 0.3, 'mlp__early_stopping': True, 'mlp__hidden_layer_sizes': (200,), 'mlp__n_iter_no_change': 20, 'mlp__solver': 'adam', 'mlp__validation_fraction': 0.1, 'select_k_best__k': 29}
Optimized cross-validation score: 0.37

Processing file: features_2.csv
Optimized parameters for MLP: {'m